## Sensitive analysis:


In [1]:
from sast.sast import *
from sktime.datasets import load_UCR_UEA_dataset, tsc_dataset_names
from sktime.classification.kernel_based import RocketClassifier
import time
import pandas as pd


/home/nirojasvar/anaconda3/envs/rsast_env/lib/python3.10/site-packages/mass_ts/_mass_ts.py:17: UserWarning: GPU support will not work. You must pip install mass-ts[gpu].
  warnings.warn(


### Select Datasets for hypertunning RSAST

It is runned RSAST in a set of UCR datasets with a predefined number of runs ("runs"). Then, it is selected a range ("range_total") between 1, 10, 100 and 10000 for the selected dataset.

In [16]:
ds_sens = tsc_dataset_names.univariate_equal_length
#ds_sens = ['ACSF1', 'Adiac', 'ArrowHead', 'Beef', 'BeetleFly', 'BirdChicken', 'BME', 'Car', 'CBF', 'Chinatown', 'ChlorineConcentration', 'CinCECGTorso', 'Coffee']
ds_sens = ['Chinatown']
max_ds=len(ds_sens) #exploring dataset in UEA & UCR Time Series Classification Repository
#print(ds_sens)
print(ds_sens)


['Chinatown']


In [31]:
runs = 5
range_total = [1,10,30,50,100,1000]
#list
df_result = {}
list_score = []
list_overall_time = []
list_cweight_time = []
list_fsubsequence_time = []
list_tdataset_time = []
list_tclassifier_time = []
list_dataset = []
list_hyperparameter = []
list_method = []
list_rpoint = []
list_nb_per_class = []
list_len_method = []

for ds in ds_sens:
    print("ds="+ds)
    X_train, y_train = load_UCR_UEA_dataset(name=ds, extract_path='data', split="train", return_type="numpy2d")
    X_test, y_test = load_UCR_UEA_dataset(name=ds, extract_path='data', split="test", return_type="numpy2d")
    for i in range(runs):
        for p in range_total:
            for g in range_total:
                if p ==0:
                    p=1
                if g ==0:
                    g=1
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="both",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))

                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)

                list_dataset.append(ds)
                list_hyperparameter.append("ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("ACF&PACF")
                print("ACF&PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="ACF",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)

                list_dataset.append(ds)
                list_hyperparameter.append("ACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("ACF")
                print("ACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="PACF",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)

                list_dataset.append(ds)
                list_hyperparameter.append("PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("PACF")
                print("PACF: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                start = time.time()
                random_state = None
                rsast_ridge = RSAST(n_random_points=p,nb_inst_per_class=g, len_method="None",classifier=RidgeClassifierCV())
                rsast_ridge.fit(X_train, y_train)
                end = time.time()
                list_score.append(rsast_ridge.score(X_test,y_test))
                
                list_overall_time.append(end-start)
                list_cweight_time.append(rsast_ridge.time_calculating_weights)
                list_fsubsequence_time.append(rsast_ridge.time_creating_subsequences)
                list_tdataset_time.append(rsast_ridge.transform_dataset)
                list_tclassifier_time.append(rsast_ridge.time_classifier)
                
                list_dataset.append(ds)
                list_hyperparameter.append("None: n_random_points="+str(p)+" nb_inst_per_class="+str(g))
                list_rpoint.append(str(p))
                list_nb_per_class.append(str(g))
                list_method.append("Rsast")
                list_len_method.append("None")
                print("None: n_random_points="+str(p)+" nb_inst_per_class="+str(g))

df_result['accuracy']=list_score

df_result['time']=list_overall_time
df_result['cweights_time']=list_cweight_time
df_result['fsubsequence_time']=list_fsubsequence_time
df_result['tdataset_time']=list_tdataset_time
df_result['tclassifier_time']=list_tclassifier_time


df_result['dataset_name']=list_dataset
df_result['classifier_name']=list_hyperparameter
df_result['rpoint']=list_rpoint
df_result['nb_per_class']=list_nb_per_class
df_result['method']=list_method
df_result['len_method']=list_len_method

df_result=pd.DataFrame(df_result)
# export a overall dataset with results
df_result.to_csv("df_overall_tunning.csv") 

ds=Chinatown
ACF&PACF: n_random_points=1 nb_inst_per_class=1
ACF: n_random_points=1 nb_inst_per_class=1
PACF: n_random_points=1 nb_inst_per_class=1
None: n_random_points=1 nb_inst_per_class=1
ACF&PACF: n_random_points=1 nb_inst_per_class=10
ACF: n_random_points=1 nb_inst_per_class=10
PACF: n_random_points=1 nb_inst_per_class=10
None: n_random_points=1 nb_inst_per_class=10
ACF&PACF: n_random_points=1 nb_inst_per_class=100
ACF: n_random_points=1 nb_inst_per_class=100
PACF: n_random_points=1 nb_inst_per_class=100
None: n_random_points=1 nb_inst_per_class=100
ACF&PACF: n_random_points=10 nb_inst_per_class=1
ACF: n_random_points=10 nb_inst_per_class=1
PACF: n_random_points=10 nb_inst_per_class=1
None: n_random_points=10 nb_inst_per_class=1
ACF&PACF: n_random_points=10 nb_inst_per_class=10
ACF: n_random_points=10 nb_inst_per_class=10
PACF: n_random_points=10 nb_inst_per_class=10
None: n_random_points=10 nb_inst_per_class=10
ACF&PACF: n_random_points=10 nb_inst_per_class=100
ACF: n_random_poi

In [32]:
df_result=pd.read_csv("df_overall_tunning.csv")

In [33]:
# create and export a pivot table with the mean of score
pivot_table = pd.pivot_table(df_result, values='accuracy', index=['classifier_name','dataset_name'], aggfunc='mean')
pivot_table.to_csv("df_pivot_tunning_hyper.csv") 


In [34]:
pd.pivot_table(df_result, values='accuracy', index=['len_method'],columns=['nb_per_class','rpoint'], aggfunc='mean')

nb_per_class       1                             10                       \
rpoint             1         10        100       1         10        100   
len_method                                                                 
ACF           0.751603  0.906122  0.899708  0.929446  0.967347  0.981341   
ACF&PACF      0.848397  0.946939  0.946939  0.899125  0.980758  0.984257   
None          0.849563  0.922449  0.861224  0.946356  0.973178  0.976093   
PACF          0.801166  0.872303  0.942857  0.960350  0.980758  0.983090   

nb_per_class       100                      
rpoint             1         10        100  
len_method                                  
ACF           0.954519  0.971429  0.979592  
ACF&PACF      0.944023  0.976093  0.981924  
None          0.940525  0.963848  0.977259  
PACF          0.951020  0.975510  0.983090

In [35]:
# create a pivot table with the mean of score by hyperparameter
pd.pivot_table(df_result, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.812682,0.933819,0.947522
10,0.911953,0.975510,0.971720
100,0.912682,0.981195,0.980466


In [36]:
# filter by method with ACF&PACF
# create a pivot table with the mean of score by hyperparameter
df_result_acc=df_result[df_result["len_method"]=="ACF&PACF"]
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.848397,0.899125,0.944023
10,0.946939,0.980758,0.976093
100,0.946939,0.984257,0.981924


In [37]:
# create a pivot table with the variatioon of score by hyperparameter
pd.pivot_table(df_result_acc, values='accuracy', index=['rpoint'],columns=['nb_per_class'], aggfunc='var')

nb_per_class,1,10,100
rpoint,,,
1,0.011556,0.002047,0.000860
10,0.000384,0.000020,0.000023
100,0.000924,0.000003,0.000006


#### Analysis of time complexity

##### Overall time complexity

In [38]:
# create a pivot table with the mean of time spent in minutes
pd.pivot_table(df_result_acc, values='time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.035733,0.064551,0.065056
10,0.051483,0.134778,0.151523
100,0.112825,0.907014,0.906635


##### Calculate weights time complexity

In [39]:
pd.pivot_table(df_result_acc, values='cweights_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.006274,0.005948,0.006123
10,0.012165,0.010064,0.006988
100,0.006030,0.008874,0.007511


##### Finding subsequences time complexity

In [40]:
pd.pivot_table(df_result_acc, values='fsubsequence_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.005742,0.030302,0.036330
10,0.008799,0.053819,0.055026
100,0.025177,0.233774,0.230257


##### Transform Dataset time complexity

In [41]:
pd.pivot_table(df_result_acc, values='tdataset_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.021197,0.012470,0.012308
10,0.020159,0.066478,0.080071
100,0.075172,0.659409,0.659667


##### Classifier time complexity

In [42]:
pd.pivot_table(df_result_acc, values='tclassifier_time', index=['rpoint'],columns=['nb_per_class'], aggfunc='mean')

nb_per_class,1,10,100
rpoint,,,
1,0.001901,0.015479,0.009955
10,0.009999,0.003813,0.008056
100,0.005707,0.003417,0.007890
